In [6]:
source_urls = ["https://sunnyvaleca.legistar.com/Calendar.aspx"]

In [7]:
import urllib
from bs4 import BeautifulSoup

In [8]:
def read_page(url):
    with urllib.request.urlopen(url) as f:
        page = f.read()
    return BeautifulSoup(page)

In [66]:
import re
def get_site_url(url):
    return re.match("([^/]*//[^/]*/)", url).groups()[0]

In [69]:
def get_link(a, url):
    href = a.get("href")
    if not href:
        return None
    # ingore empty URLs
    if href == "":
        return None
    # ignore javascript calls (TODO: is this correct?)
    if re.match("javascript", href):
        return None
        return None
    # ignore anchors
    if href[0] == "#":
        return None
    site_url = get_site_url(url)
    # if it's a full url AND a subpage of this site, return the url
    if re.match(site_url, href):
        return href
    # but if it's a full url from some other site, ignore it
    if re.match("https?://", href):
        return None
    return "{}{}".format(site_url, href)
    
links_from_this_page = set([get_link(a, source_url) for a in page.find_all("a") if get_link(a, source_url)])
# links_from_this_page

In [103]:
urls_searched = set()
urls_to_search = set([source_url])
t = 0
while len(urls_to_search) > 0 and t < 5:
# while len(urls_to_search) > 0:
    url = urls_to_search.pop()
    urls_searched.add(url)
    print("reading page: {}".format(url))
    with urllib.request.urlopen(url) as f:
        contents = f.read()
    if re.search(b'DOCTYPE html', contents):
        print("it's an html page, so we're searching it.")
        page = BeautifulSoup(contents)
        links_from_this_page = set([get_link(a, source_url) for a in page.find_all("a") if get_link(a, source_url)])
        new_links_to_search = links_from_this_page.difference(urls_searched).difference(urls_to_search)
        urls_to_search.update(new_links_to_search)
        print("{} new links found, {} total read, {} still need to be read".format(
            len(new_links_to_search),
            len(urls_searched),
            len(urls_to_search)))
    else:
        print("it's not an html page, so we're just keeping track of the url")
        print("{} total read, {} still need to be read".format(
            len(new_links_to_search),
            len(urls_searched),
            len(urls_to_search)))
    print("***")
    t+=1
#     links_to_search = [link for link in links_from_this_page if not link in urls_searched]
# urls_to_search


reading page: https://sunnyvaleca.legistar.com/Calendar.aspx
it's an html page, so we're searching it.
65 new links found, 1 total read, 65 still need to be read
***
reading page: https://sunnyvaleca.legistar.com/Legislation.aspx
it's an html page, so we're searching it.
0 new links found, 2 total read, 64 still need to be read
***
reading page: https://sunnyvaleca.legistar.com/View.ashx?M=A&ID=651877&GUID=CBCE9D3D-CE13-40A5-B781-1EB3305F4EBC
it's not an html page, so we're just keeping track of the url
0 total read, 3 still need to be read
***
reading page: https://sunnyvaleca.legistar.com/View.ashx?M=IC&ID=661710&GUID=602A22B1-628A-421C-8D76-CFD951F83807
it's not an html page, so we're just keeping track of the url
0 total read, 4 still need to be read
***
reading page: https://sunnyvaleca.legistar.com/MeetingDetail.aspx?ID=672131&GUID=61E92095-3E44-4BD6-A9BC-544F021B0DF7&Search=
it's an html page, so we're searching it.
25 new links found, 5 total read, 86 still need to be read
***
